# Importer les données

In [39]:
import pandas as pd
import plotly.graph_objects as go

In [40]:
df_city_light = pd.read_csv('../city_data/df_city_light.csv')
df_top_5_city = pd.read_csv('../city_data/df_top_5_city.csv')
df_hotels = pd.read_csv("../hotel_data/booking_hotels_meteo.csv")

# Obtenir la carte des meilleures villes

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scattermap(
    lat=df_city_light['latitude'],
    lon=df_city_light['longitude'],
    mode='markers+text',
    marker=go.scattermap.Marker(
        size=df_city_light['global_score'],
        color=df_city_light['global_score'],
        colorscale='Viridis',
        cmin=min(df_city_light['global_score']),
        cmax=20,  
        showscale=True,
        sizemode='area',
        sizeref=2.*max(df_city_light['global_score'])/(40.**2),
        sizemin=10,
    ),
    text=df_city_light['rank'],  # Rang + nom de la ville quand on passe la souris
    hoverinfo='text'
))

for i, line in df_top_5_city.iterrows():
    fig.add_trace(go.Scattermap(
        lat=[line['latitude']],
        lon=[line['longitude']],
        mode='markers+text',
        marker=go.scattermap.Marker(
            size=df_city_light['global_score']/30,
            color='black',
            cmin=1,
            cmax=20,  
            showscale=False,
            sizemode='area',
            sizeref=2.*max(df_city_light['global_score'])/(40.**2),
            sizemin=1,
        ),
        hoverinfo='skip',
        showlegend=False
    ))

fig.update_layout(
    map_style="carto-positron",
    map_zoom=4.4,
    map_center={"lat": 46.8, "lon": 2.5},
    margin={"r":0,"t":0,"l":0,"b":0}
)

fig.show()

#Sauvegarder en HTML
fig.write_html("../maps/carte_villes_interactive.html")

# Obtenir la carte des meilleurs hotels

In [ ]:
import plotly.graph_objects as go

#Préparer les données : regrouper les hôtels par ville
df_cities = df_hotels.groupby('city').first().reset_index()

#Créer le texte de survol avec les 5 meilleurs hôtels par ville
hover_texts = []
for city in df_cities['city']:
    #Récupérer les 5 hôtels de cette ville
    hotels_in_city = df_hotels[df_hotels['city'] == city].head(5)
    
    #Créer la liste des hôtels
    hotel_list = f"<b>{city}</b><br><br>"
    for i, (idx, hotel) in enumerate(hotels_in_city.iterrows(), 1):
        hotel_list += f"{i}. {hotel['name']}<br>"
    
    hover_texts.append(hotel_list)

df_cities['hover_text'] = hover_texts

#Créer la figure
fig = go.Figure()

#Afficher les hôtels et l'emplacement des villes
fig.add_trace(go.Scattermap(
    lat=df_cities['latitude'],
    lon=df_cities['longitude'],
    mode='markers',
    marker=dict(
        size=40,
        color='rgba(255, 99, 71, 0.8)',
    ),
    text=df_cities['hover_text'],
    hovertemplate='%{text}<extra></extra>',
    showlegend=False
))

#Petit point au centre
fig.add_trace(go.Scattermap(
    lat=df_cities['latitude'],
    lon=df_cities['longitude'],
    mode='markers',
    marker=dict(
        size=8,
        color='rgba(100, 149, 237, 1)',
    ),
    hoverinfo='skip',
    showlegend=False
))

fig.update_layout(
    map_style="carto-positron",
    map_zoom=4.4,
    map_center={"lat": 46.8, "lon": 2.5},
    margin={"r":0,"t":0,"l":0,"b":0}
)

fig.show()

#Sauvegarder en HTML
fig.write_html("../maps/carte_hotels_interactive.html")